Notebook to perform numerical integrations and fits of the analytical solutions for the LL model.
Results are saved in the data directory and loaded when producing the plots.

### Imports

In [1]:
import numpy as np
from scipy.optimize import OptimizeResult
from tqdm import tqdm

import load_data_utils
import LL_utils, analyze_data_utils

## 1. Equilibrium results

### One particle entanglement entropy from LL analytical result for one body density

In [2]:
ϵ = 0.84 
dV = 0.05
V_array = np.arange(-2.0,2.0+dV/2,dV)

ias = [0,1,4]  
αs =  [1,2,5]
 
for i, (ia, α) in enumerate(zip(ias,αs)):
    filename = f'../data/eq_LL/entropiesLL_a{α}_LL_tdlimit_eps{ϵ}.npz' 
 
    A_i_LL = []
    γ_LL = []
    V_LL = []
    for γ,v  in zip(tqdm(LL_utils.γeq_V(V_array)),V_array): 
        try: 
            A_i_LL.append(LL_utils.Renyi_inf(LL_utils.fq_inf,args=(γ,ϵ),α=α,ϵ=ϵ, limits=(-15,15)))
            γ_LL.append(γ)
            V_LL.append(v)
        except ValueError:
            pass  
    
    np.savez(filename,gamma=γ_LL, A=A_i_LL, V=V_LL)    

c:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\PhD Thesis\Material\TN_Project\GitHub\DelMaestroGroup\papers-code-OneParticleEntanglementEntropy\src\LL_utils.py:43: RuntimeWarning: invalid value encountered in arccos
  return π/(2*np.arccos(-V/2))
c:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\PhD Thesis\Material\TN_Project\GitHub\DelMaestroGroup\papers-code-OneParticleEntanglementEntropy\src\LL_utils.py:43: RuntimeWarning: divide by zero encountered in true_divide
  return π/(2*np.arccos(-V/2))
  0%|          | 0/81 [00:00<?, ?it/s]c:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\PhD Thesis\Material\TN_Project\GitHub\DelMaestroGroup\papers-code-OneParticleEntanglementEntropy\src\LL_utils.py:76: RuntimeWarning: invalid value encountered in sin
  c2 = 2 * Γ(-γeq**2)*np.sin(π*γeq**2/2) / (4*kF*π)
 16%|█▌        | 13/81 [00:05<00:36,  1.86it/s]c:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\PhD Thesis\Material\TN_Project\GitHub\DelMaestroGroup\papers-code-OneParticleEn

### Fit LL result to equilibrium one particle entanglement in the thermodynamic limit from finite size scaling

Load numerical data to fit to

In [3]:
dV = 0.05
V_array = np.arange(-2.0,2.0+dV/2,dV)
N_array_ED = np.arange(2,17.5,1)
nVs = np.shape(V_array)[0]

Sα_ED = np.zeros((np.size(V_array),11,np.size(N_array_ED)))
# ED results N=2-16
idV = 5
nFermions_array = np.array(range(2,16+1))
dat_path_template = "../data/eq_ED/particle_entanglement_n01_M{M:02d}_N{N:02d}_t+1.000_Vp+0.000_Vsta-2.000_Vend+2.000_Vstp+0.010.dat"
replace_list = [dict(M=2*n,N=n) for n in nFermions_array]
_,_, S_ld = load_data_utils.load_entanglement_from_files(dat_path_template, replace_list) 
Sα_ED[:,:,:np.size(nFermions_array)] = S_ld[::idV,:,:np.shape(S_ld)[2]]
# ED results N=17-19 
nFermions_array = np.array([17,19])
dat_path_template = "../data/eq_ED/particle_entanglement_n01_M{M:02d}_N{N:02d}_t+1.000_Vp+0.000_Vsta-2.000_Vend+2.000_Vstp+0.050.dat"
replace_list = [dict(M=2*n,N=n) for n in nFermions_array]
_,V_ld, S_ld = load_data_utils.load_entanglement_from_files(dat_path_template, replace_list)  
indV = np.argsort(V_ld[:,0])
Sα_ED[:,:,-np.size(nFermions_array):] = S_ld[indV,:,:]


N_array_DMRG = np.concatenate([np.arange(17,26,1),[30,40,50]])
Sα_DMRG = np.zeros((np.size(V_array),11,np.size(N_array_DMRG)))
# DMRG results N=17-25,30,40,50 
nFermions_array_1 = np.concatenate([np.arange(17,26,1),[30,40,50]]) 
dat_path_template = "../data/eq_DMRG/particle_entanglement_n01_M{M:02d}_N{N:02d}_t+1.000_Vp+0.000_Vsta-2.000_Vend+2.000_Vnum0081.dat"
replace_list = [dict(M=2*n,N=n) for n in nFermions_array_1]
_,V_ld, S_ld = load_data_utils.load_entanglement_from_files(dat_path_template, replace_list)
indV = np.argsort(V_ld[:,0]) 
Sα_DMRG[:,:,:np.size(nFermions_array_1)] = S_ld[indV,:,:]  
 

N_all = np.concatenate([N_array_ED,N_array_DMRG[N_array_DMRG > np.max(N_array_ED)]],axis=-1)
S_all = np.concatenate([Sα_ED[:,:,:],Sα_DMRG[:,:,N_array_DMRG > np.max(N_array_ED)]],axis=-1)

Sinf = np.zeros((np.shape(S_all)[0],11))
for i,ia in enumerate([1,2,3,4,5,6,7,8,9,10,0.5]): 
    Sinf[:,i] = analyze_data_utils.interpolate_entropy_tdlimit(S_all, N_all, iα = ia, fit_npoints= 10,V_array=V_array)

Fit data

In [4]:
filename = '../data/eq_LL/cutoff_optimization.npz'

dV = 0.05
V_array = np.arange(-2.0,2.0+dV/2,dV)

A_1 = Sinf[:,0]

V_o = V_array[A_1 != np.nan][np.abs(V_array) > 1e-10]
A_1 = A_1[A_1 != np.nan][np.abs(V_array) > 1e-10]

A_theory = lambda ϵ, γeq: LL_utils.Renyi_inf(fqbar=LL_utils.fq_inf,args=(γeq,ϵ),α=1,ϵ=ϵ, limits=[-15,15])
ϵ_opt = analyze_data_utils.optimize_ϵ_toED(numerics = A_1, theory_fnc = A_theory, theory_args_list = [(LL_utils.γeq_V(v),) for v in V_o], start_val=0.5, optim_kwag=dict(tol=1e-4), no_dynamic_start_val=True  )

np.savez(filename,A=A_1,V=V_o,eps = ϵ_opt)

V_fit = []
ϵ_fit = []
for v, optref in zip(V_o,ϵ_opt): 
    if isinstance(optref,OptimizeResult):
        V_fit.append(v)
        ϵ_fit.append(optref.x)
V_fit = np.array(V_fit)
ϵ_fit = np.array(ϵ_fit)

c:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\PhD Thesis\Material\TN_Project\GitHub\DelMaestroGroup\papers-code-OneParticleEntanglementEntropy\src\LL_utils.py:43: RuntimeWarning: divide by zero encountered in double_scalars
  return π/(2*np.arccos(-V/2))
  0%|          | 0/80 [00:00<?, ?it/s]c:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\PhD Thesis\Material\TN_Project\GitHub\DelMaestroGroup\papers-code-OneParticleEntanglementEntropy\src\analyze_data_utils.py:89: OptimizeWarning: Unknown solver options: maxiter, disp
  res.append(scipy.optimize.root(fun=opt_fnc, x0=[start_val], args=args, options=dict(maxiter=200,disp=False),**optim_kwag))
c:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\PhD Thesis\Material\TN_Project\GitHub\DelMaestroGroup\papers-code-OneParticleEntanglementEntropy\src\LL_utils.py:126: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  i

LL one particle entanglement entropy for equilibrium fitted cutoff

In [5]:
ias = [0,1,4]  
αs =  [1,2,5]

for i, (ia, α) in enumerate(zip(ias,αs)):
    filename = f'../data/eq_LL/entropiesLL_a{α}_LL_tdlimit_eps_optS1.npz' 

    A_i_LL = []
    γ_LL = []
    V_LL = []
    for γ,v,ϵo  in zip(tqdm(LL_utils.γeq_V(V_fit)),V_fit,ϵ_fit): 
        try: 
            ϵo = ϵo[0]
            A_i_LL.append(LL_utils.Renyi_inf(LL_utils.fq_inf,args=(γ,ϵo),α=α,ϵ=ϵo, limits=(-15,15)))
            γ_LL.append(γ)
            V_LL.append(v)
        except ValueError:
            pass  
        
    np.savez(filename,gamma=γ_LL, A=A_i_LL, V=V_LL)    

100%|██████████| 78/78 [00:40<00:00,  1.92it/s]


## 2. Quench results

### Plateaus of LL time evolution

Plateau values

In [6]:
v  = 1.0
ϵ = 0.84

L_array = np.arange(6,50,4)

 
for iL, L in enumerate(L_array): 
     
    filename = f'../data/quench_LL/LL_time_V{v:4.3f}_Lseries_L{L:02d}_eps{ϵ:4.3f}.npz'

    tres_array = np.arange(0,L/2,0.1) 
    γ = LL_utils.γ_V(v)

    A1_t = []
    tress = []  
    A_free = LL_utils.Renyi_fin(LL_utils.fq_fin_t,args=(0,ϵ,L,0), α=1,ϵ=ϵ,L=L, limits=(-2*L,2*L))
    for tres in tqdm(tres_array):  
        
        try: 
            nrm = LL_utils.check_norm_fin(LL_utils.fq_fin_t,(γ,ϵ,L,tres),ϵ,L,[-2*L,2*L])
            f_q_normed = lambda _q, _γ,_ϵ,_L,_tres : LL_utils.fq_fin_t(_q, _γ,_ϵ,_L,_tres)/nrm
            A1_t.append(LL_utils.Renyi_fin(f_q_normed,args=(γ,ϵ,L,tres), α=1,ϵ=ϵ,L=L, limits=(-2*L,2*L))) 
            tress.append(tres) 
        except ValueError:
            pass  
    
    np.savez(filename,tres=tress, A=A1_t-A_free, V=v)  

 64%|██████▎   | 70/110 [06:15<03:38,  5.47s/it]

Scaling of plateaus

In [ ]:
Vs = np.arange(-2.0,2.0,0.1)

ϵ = 0.84

L_array = np.arange(6,58+4,8)

for v in Vs:
    filename = f'../data/quench_LL/LL_time_V{v:4.3f}_L_series_Lst{L_array[0]}_Lend{L_array[-1]}_plateaus_eps{ϵ:4.3f}.npz'
    
    tress = []
    A1_plateau = [] 
    for iL, L in enumerate(tqdm(L_array)): 
        tres = L/4 
        γ = LL_utils.γ_V(v)

        A_free = LL_utils.Renyi_fin(LL_utils.fq_fin_t,args=(0,ϵ,L,0), α=1,ϵ=ϵ,L=L, limits=(-2*L,2*L)) 
        try: 
            nrm = LL_utils.check_norm_fin(LL_utils.fq_fin_t,(γ,ϵ,L,tres),ϵ,L,[-2*L,2*L])
            f_q_normed = lambda _q, _γ,_ϵ,_L,_tres : LL_utils.fq_fin_t(_q, _γ,_ϵ,_L,_tres)/nrm
            A1_plateau.append(LL_utils.Renyi_fin(f_q_normed,args=(γ,ϵ,L,tres), α=1,ϵ=ϵ,L=L, limits=(-2*L,2*L))-A_free) 
            tress.append(tres) 
        except ValueError:
            pass  
        
        np.savez(filename,L_array=L_array,tres=tress, A=A1_plateau, V=v)  

  0%|          | 0/7 [00:00<?, ?it/s]<ipython-input-6-f36510eee64d>:19: RuntimeWarning: invalid value encountered in double_scalars
  f_q_normed = lambda _q, _γ,_ϵ,_L,_tres : LL_utils.fq_fin_t(_q, _γ,_ϵ,_L,_tres)/nrm
 71%|███████▏  | 5/7 [00:16<00:07,  3.75s/it]c:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\PhD Thesis\Material\TN_Project\GitHub\DelMaestroGroup\papers-code-OneParticleEntanglementEntropy\src\LL_utils.py:106: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  if α == 1:
100%|██████████| 7/7 [01:57<00:00, 16.78s/it]


### Steady state LL result

In [ ]:
ϵ = 0.84

Vs = np.arange(-2.0,2.0,0.1)
αs = [1] 
imap = {0:0}
A_i_LL = [[] for _ in αs]
γ_LL = [[] for _ in αs]
V_LL= [[] for _ in αs]

# A_i_LL[i] with i = 0 negativity, and α = i otherwise
for i, α in enumerate(αs): 
    filename = f'../data/quench_LL/entropiesLL_a{α}_LL_tdlimit_tinf_eps{ϵ}_pow1.npz' 

    for γ,v  in zip(tqdm(LL_utils.γ_V(Vs)),Vs): 
        try: 
            A_i_LL[i].append(LL_utils.Renyi_inf(LL_utils.fq_inf,args=(γ,ϵ),α=α,ϵ=ϵ, limits=(-15,15)))
            γ_LL[i].append(γ)
            V_LL[i].append(v)
        except ValueError:
            pass  
        
    np.savez(filename,gamma=γ_LL[i], A=A_i_LL[i], V=V_LL[i])    
  

100%|██████████| 40/40 [00:22<00:00,  1.76it/s]


### Thermodynamic limit steady state result

In [ ]:
ϵ = 0.84

ias = [0,1,2,4]
αs = [1,2,3,5]

dV = 0.05
V_array = np.arange(-2.0,2.0+dV/2,dV)
 
for i, (ia, α) in enumerate(zip(ias,αs)):
    filename = f'../data/quench_LL/entropiesLL_quench_a{α}_LL_tdlimit_eps{ϵ}.npz' 

    A_i_LL = []
    γ_LL = []
    V_LL = []
    for γ,v  in zip(tqdm(LL_utils.γ_V(V_array)),V_array): 
        try:   
            A_i_LL.append(LL_utils.Renyi_inf(LL_utils.fq_inf,args=(γ,ϵ),α=α,ϵ=ϵ, limits=(-15,15)))
            γ_LL.append(γ)
            V_LL.append(v)
        except ValueError:
            pass  
        
    np.savez(filename,gamma=γ_LL, A=A_i_LL, V=V_LL)    

100%|██████████| 81/81 [00:35<00:00,  2.31it/s]


### Fit LL result to quench steady state one particle entanglement in the thermodynamic limit from finite size scaling

Load data

In [ ]:
#ED
data_folder = "../data/quench_ED/"

V_array_quench = np.arange(-1.9,1.95,0.1)
N_array_EDquench = np.arange(2,13,1)

i_N = lambda n: n-N_array_EDquench[0]

# load
quench_N = []
for N in N_array_EDquench:
    quench_N.append(load_data_utils.load_quench_data(N,V_array_quench, folder=data_folder))
# estimate t->inf limit
for n in N_array_EDquench:
    quench_N[i_N(n)] = analyze_data_utils.estimate_tinf_limit_all(quench_N[i_N(n)],n)
# estimate TD limit
V_quench, Sinf_quench, Serr_quench = analyze_data_utils.estimate_tinf_tdlimit_all(quench_N,N_array_EDquench,fit_npoints=5)

Binning Converge Error: no plateau found
Binning Converge Error: no plateau found
Binning Converge Error: no plateau found
Binning Converge Error: no plateau found
Binning Converge Error: no plateau found
Binning Converge Error: no plateau found
Binning Converge Error: no plateau found
Warning for V=-1.9, iR=0 error estimation failed.  Use fallback method.
Warning for V=-1.9, iR=1 error estimation failed.  Use fallback method.
Warning for V=-1.9, iR=2 error estimation failed.  Use fallback method.
Warning for V=-1.9, iR=3 error estimation failed.  Use fallback method.
Warning for V=-1.9, iR=4 error estimation failed.  Use fallback method.
Warning for V=-1.9, iR=5 error estimation failed.  Use fallback method.
Warning for V=-1.9, iR=6 error estimation failed.  Use fallback method.
Warning for V=-1.9, iR=7 error estimation failed.  Use fallback method.
Warning for V=-1.9, iR=8 error estimation failed.  Use fallback method.
Warning for V=-1.9, iR=9 error estimation failed.  Use fallback m

c:\Users\mtham\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\mtham\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\mtham\anaconda3\lib\site-packages\numpy\core\_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\mtham\anaconda3\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
c:\Users\mtham\anaconda3\lib\site-packages\numpy\core\_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Binning Converge Error: no plateau found
Warning for V=-1.9, iR=0 error estimation failed.  Use fallback method.
Binning Converge Error: no plateau found
Warning for V=-1.9, iR=1 error estimation failed.  Use fallback method.
Binning Converge Error: no plateau found
Warning for V=-1.9, iR=2 error estimation failed.  Use fallback method.
Binning Converge Error: no plateau found
Warning for V=-1.9, iR=3 error estimation failed.  Use fallback method.
Binning Converge Error: no plateau found
Warning for V=-1.9, iR=4 error estimation failed.  Use fallback method.
Binning Converge Error: no plateau found
Warning for V=-1.9, iR=5 error estimation failed.  Use fallback method.
Binning Converge Error: no plateau found
Warning for V=-1.9, iR=6 error estimation failed.  Use fallback method.
Binning Converge Error: no plateau found
Warning for V=-1.9, iR=7 error estimation failed.  Use fallback method.
Binning Converge Error: no plateau found
Warning for V=-1.9, iR=8 error estimation failed.  Use 

Add date where we have DMRG results

In [ ]:
dmrg_data_N_lookup =  {0.2: [11,12,13,14,15], -0.2: [13,14], 0.9: [11,12,13,14,15], -0.9: [12,13,14], 1.3: [13,14], 1.7: [13,14] , -1.3:[12,13], -1.7: [13]}

for i in range(1,12):
    for v in dmrg_data_N_lookup.keys():
        # ED Data
        N_array_fit = N_array_EDquench
        SinfN_fit = np.array([quench("S",i=i,t=[-1],V=v)[0] for quench in quench_N]) 
        SerrN_fit = np.array([quench("Serr",i=i,V=v) for quench in quench_N]) 
        iv = np.where(np.abs(V_quench-v)<1e-8)[0][0]
        # DMRG Data
        for nn in dmrg_data_N_lookup[v]:
            quench_DMRG = load_data_utils.load_itensor_quench_data(nn,tend=40,tstep=0.01,Vsta=v,Vend=v,Vnum=1,folder="../data/quench_DMRG") 
            quench_DMRG  = analyze_data_utils.estimate_tinf_limit_all(quench_DMRG ,nn) 
            if not nn in N_array_fit:
                    N_array_fit = np.concatenate([N_array_fit,[nn]])
                    SinfN_fit  = np.concatenate([SinfN_fit,[quench_DMRG("S",i=i,t=[-1],V=v)[0]]])
                    SerrN_fit = np.concatenate([SerrN_fit,[quench_DMRG("Serr",i=i,V=v)]])
        a, S_tdlimit, S_tdlimit_err = analyze_data_utils.fit_tail_linear(1/N_array_fit,SinfN_fit,npoints=6,std_err=SerrN_fit)

        Sinf_quench[i-1,[iv]] = S_tdlimit
        Serr_quench[i-1,[iv]] = S_tdlimit_err

Warning for V=-1.3, iR=0 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=1 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=2 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=3 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=4 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=5 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=6 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=7 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=8 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=9 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=10 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=0 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=1 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=2 error estimation failed.  Use fallback

c:\Users\mtham\anaconda3\lib\site-packages\numpy\lib\function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)


Warning for V=-1.3, iR=0 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=1 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=2 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=3 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=4 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=5 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=6 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=7 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=8 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=9 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=10 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=0 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=1 error estimation failed.  Use fallback method.
Warning for V=-1.3, iR=2 error estimation failed.  Use fallback

Fit results

In [ ]:
filename = '../data/quench_LL/cutoff_optimization_quench.npz'
A_1 = Sinf_quench[0,:]
 
V_o = V_array_quench[A_1 != np.nan][np.abs(V_array_quench) > 1e-10]
A_1 = A_1[A_1 != np.nan][np.abs(V_array_quench) > 1e-10]

A_theory = lambda ϵ, γ: LL_utils.Renyi_inf(fqbar=LL_utils.fq_inf,args=(γ,ϵ),α=1,ϵ=ϵ, limits=[-15,15])
ϵ_opt = analyze_data_utils.optimize_ϵ_toED(numerics = A_1, theory_fnc = A_theory, theory_args_list = [(LL_utils.γ_V(v),) for v in V_o], start_val=0.5, optim_kwag=dict(tol=1e-4), no_dynamic_start_val=True  )

np.savez(filename,A=A_1,V=V_o,eps = ϵ_opt)

V_fit_quench = []
ϵ_fit_quench = []
for v, optref in zip(V_o,ϵ_opt): 
    if isinstance(optref,OptimizeResult):
        V_fit_quench.append(v)
        ϵ_fit_quench.append(optref.x)
V_fit_quench = np.array(V_fit_quench)
ϵ_fit_quench = np.array(ϵ_fit_quench)

  0%|          | 0/38 [00:00<?, ?it/s]c:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\PhD Thesis\Material\TN_Project\GitHub\DelMaestroGroup\papers-code-OneParticleEntanglementEntropy\src\analyze_data_utils.py:89: OptimizeWarning: Unknown solver options: maxiter, disp
  res.append(scipy.optimize.root(fun=opt_fnc, x0=[start_val], args=args, options=dict(maxiter=200,disp=False),**optim_kwag))
c:\Users\mtham\Dropbox\Dokumente\Universität\Skripte\PhD Thesis\Material\TN_Project\GitHub\DelMaestroGroup\papers-code-OneParticleEntanglementEntropy\src\LL_utils.py:126: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose i

Compute entanglement with fitted cutoff

In [ ]:
ias = [0,1,2,4]
αs = [1,2,3,5]

for i, (ia, α) in enumerate(zip(ias,αs)):
    filename = f'../data/quench_LL/entropiesLL_quench_a{α}_LL_tdlimit_eps_optS1.npz' 

    A_i_LL = []
    γ_LL = []
    V_LL = []
    for γ,v,ϵo  in zip(tqdm(LL_utils.γ_V(V_fit_quench)),V_fit_quench,ϵ_fit_quench): 
        try: 
            ϵo = ϵo[0]
            A_i_LL.append(LL_utils.Renyi_inf(LL_utils.fq_inf,args=(γ,ϵo),α=α,ϵ=ϵo, limits=(-15,15)))
            γ_LL.append(γ)
            V_LL.append(v)
        except ValueError:
            pass  
    
    np.savez(filename,gamma=γ_LL, A=A_i_LL, V=V_LL)    

100%|██████████| 38/38 [00:18<00:00,  2.11it/s]
